This file will be used mainly for the Coursera Applied Data Science Capstone

In [8]:
# import stuff

import pandas as pd
import numpy as np


ModuleNotFoundError: No module named 'pandas'

In [ ]:
df = pd.read_csv(r"C:\\Users\\mizog\\Documents\\Coursera\\Applied Data Specialization\\Applied Data Science Capstone")
df.head()